In [29]:
import pandas as pd
import numpy as np
import dash
import dash_core_components as dcc
import dash_html_components as html
import dash_table
from dash import Dash, dcc, html, Input, Output, State, callback
from iterational_solution import IterationalProblem

In [10]:
data = pd.read_json("dataset.json")
data_to_see = data.iloc[10]
data_to_see

stations          {'Златоуст (1)': ['0', '2', '1', '29', '13', '...
full_timetable    {'667': {'route': ['1', '3', '2'], 'free_carri...
Name: 10, dtype: object

In [2]:
problem = IterationalProblem("dataset.json")
problem.solve()

[{'Златоуст':    trains  number of cars arrival time departure time
  0     853              21        02:00          02:38
  1     801              34        03:24          04:22
  2     563              33        03:43          04:26
  3     182              15        02:44          03:14
  4     643               2        03:23          03:48
  5     743               0        12:45          13:00
  6     901               0        03:34          04:24
  7     405               0        02:11          03:00
  8     722               0        13:46          14:40
  9     883               0        15:51          17:02
  10    700               0        11:41          12:37,
  'Кыштым':    trains  number of cars arrival time departure time
  0     853               0        13:35          13:58
  1     801               0        14:30          15:39
  2     743               8        01:00          02:01
  3     399              12        02:00          02:33
  4     930              

In [7]:
problem.get_solution_for_city(0,'Кыштым')
df = problem.get_solution_for_city(0,'Кыштым')
df = df.rename(columns={'trains':'Поезд', 'number of cars':'Количество вагонов для сцепки',
                  'arrival time':'Время прибытия', 'departure time':'Время отбытия'}, inplace=False)
df

,Поезд,Количество вагонов для сцепки,Время прибытия,Время отбытия
0,853,0,13:35,13:58
1,801,0,14:30,15:39
2,743,8,01:00,02:01
3,399,12,02:00,02:33
4,930,22,01:48,02:10
5,658,20,01:31,03:01
6,332,9,02:51,03:28
7,617,26,01:42,03:03
8,967,0,10:25,10:42
9,692,1,11:23,12:07


In [11]:
hat = [item[:-4] for item in data_to_see['stations'].keys()]
data_to_see[1]
hat

['Златоуст',
 'Кыштым',
 'Миасс',
 'Муслюмово',
 'Челябинск',
 'Полетаево',
 'Еманжелинск']

In [12]:
from datetime import datetime, time
from datetime import datetime
timetable_values = [item['timetable'] for item in data_to_see['full_timetable'].values()]
time_range=[]
for i in range(len(timetable_values)):
    lis=[]
    for j in range(len(timetable_values[i])-1):
        x = datetime.strptime(timetable_values[i][j][-5::], '%H:%M').time()
        y = datetime.strptime(timetable_values[i][j+1][:5], '%H:%M').time()
        a = datetime.combine(datetime.min, y) - datetime.combine(datetime.min, x)
        lis.append(a)
    time_range.append(lis)
routes = [item['route'] for item in data_to_see['full_timetable'].values()]
routes_range = []
for i in range(len(routes)):
    r=[]
    for j in range(len(routes[i])-1):
        lt = f"{routes[i][j]}-{routes[i][j+1]}"
        r.append(lt)
    routes_range.append(r)
combine_dict = {}
unique_routes = set(sum(routes_range, []))
unique_timetable = set(sum(time_range, []))
combined_list=[]
for i in range(len(routes_range)):
    combine_dict.update({s:x for s,x in zip(routes_range[i], time_range[i])})
interval = {}
for key, value in combine_dict.items():
    total_seconds = value.total_seconds()
    hours = total_seconds // 3600
    minutes = (total_seconds % 3600) // 60
    interval[key] = f'{int(hours):02d}:{int(minutes):02d}'
start_time=[]
for i in range(len(timetable_values)):
    a = timetable_values[i][0][:5]
    start_time.append(a)
start_value = {}
for i, j in zip(train, start_time):
    start_value[i] = j
interval
def_data = list(interval.values())[:5]
start_time = datetime.strptime('01:10', '%H:%M').time()
def_data2 = [(datetime.strptime(t, '%H:%M') - datetime.combine(datetime.today(), start_time)) for t in def_data]
def_data2_str = [str(((dt.days * 24 + dt.seconds) // 3600) % 24 - 1).zfill(2) + ':' + str((dt.seconds // 60) % 60).zfill(2) for dt in def_data2] 
train  = ['№ '+item for item in data_to_see['full_timetable'].keys()]
dict_train = {'Поезд':train[:5], 'Время прибытия': def_data, 'Время отбытия':def_data2_str,
        'Количество вагонов для сцепки':[2,0,7,9,0],
       'Сколько вагонов везет':[12,34,28,24,4], 'Следующая станция':hat[:5]}
column_for_table = dict.keys()
data_far_table = dict.values()
df_for_table = pd.DataFrame.from_dict(dict_train) 


NameError: name 'interval' is not defined

In [1]:
app = dash.Dash(__name__)
app.layout = html.Div(
    style={'background-image': 'url("https://catherineasquithgallery.com/uploads/posts/2021-02/thumbs/1613682224_1-p-fon-dlya-prezentatsii-zheleznaya-doroga-1.jpg")',
           'background-size': 'cover',
           'width': '100%',
           'height': '110vh'},
    children=[
        html.Br(),
        html.H1(children='Координация пропуска вагонопотока',
                style={'fontSize':'36px', 'textAlign':'center'}),
        html.Div([
        html.Div([
            html.Label('Выберете экземпляр вагонопотока ', style={'fontSize':'20px', 'textAlign':'center'}),
            dcc.Input(
                id='input1',
                type='number',
                min=1,
                max=100000,
                step='1',
                style={'margin-left': '10px'}
            ),
            html.Br(),
            html.Br(),
            html.Label('Выберите станцию', style={'fontSize':'20px', 'textAlign':'center'}),
            dcc.Dropdown(hat,
                id='drop1',
                multi=False,
                style={'width': '500px'}
            ),
            html.Div(id='output1'),
            html.Br(),
            html.Button('Посмотреть вагонопоток', id='button', n_clicks=0,
                        style={'fontSize':'14px', 'width': '200px',
                               'fontWeight': 'bold', 'height': '30px'}),
        ],
                style={'width': '50%', 'float': 'left', 'height': '30vh'}
        ),
        html.Img(
        src="https://digital-natt.ru/upload/iblock/b2b/bobsqm4xuyyaf2uaqox36ly79egwvg79.png",
        style={'width': '50%', 'float': 'right'}
        )
        ]),
        html.Div([
            html.Br(),
            html.Div(
                children=[
                    html.H1(children='Выполненные расчеты',
                            style={'fontSize': '32px', 'textAlign': 'center'}),
                    dash_table.DataTable(
                        id='table',
                        data=[],
                        style_cell={'textAlign': 'left', 'minWidth': '150px',
                                    'whiteSpace': 'normal', 'height': 'auto',
                                    'fontSize': '12px'},
                        style_header={'textAlign': 'center', 'fontWeight': 'bold', 'fontSize': '14px'}
                    )

                ],
                style={'width': '100%', 'margin': '0 auto'}
            )
        ]
        )
    ]
)

@app.callback(
    Output('output1', 'children'),
    Input('drop1', 'value')
)
def update_drop(station):
    if station:
        return f'Вы сотрудник станции {station}'
    else:
        return 'Выберите станцию для расписания вагонопотока'

@app.callback(
    Output('table','data'),
    Output('table','columns'),
    State('input1', 'value'),
    State('drop1', 'value'),
    Input('button', 'n_clicks')
)

def update_table(ecz, city, click):
    if click>0:
        df = problem.get_solution_for_city((ecz-1),city)
        df = df.rename(columns={'trains':'Поезд', 'number of cars':'Количество вагонов для сцепки',
                          'arrival time':'Время прибытия', 'departure time':'Время отбытия'}, inplace=False)
        
        df1 = df.to_dict('records')
        columns = df.columns
        columns=[{"name": i, "id": i} for i in columns]
        
        return df1, columns
    else:
        return [], []

if __name__ == '__main__':
    app.run_server(mode='inline', debug=True)

NameError: name 'dash' is not defined